# Tuning Pipeline

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import FunctionTransformer
from sklearn import set_config; set_config(display='diagram')
from sklearn.pipeline import make_pipeline as make_pipe

👇 Consider the following dataset.

In [2]:
import pandas as pd

data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/08-Workflow/tuning_pipeline_data.csv")
data.head()

,games played,minutes played,points per game,field goals made,field goal attempts,field goal percent,3 point made,3 point attempt,3 point %,free throw made,free throw attempts,free throw %,offensive rebounds,defensive rebounds,rebounds,assists,steals,blocks,turnovers,target_5y
0,36.0,27.4,7.4,2.6,7.6,NaN,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0
1,35.0,26.9,NaN,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0
2,NaN,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0
3,58.0,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1
4,48.0,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1


- Each observations represents a player
- Each column a characteristic of performance

The target defines whether the player lasted less than 5 years (`0`) vs. 5 years or more (`1`) as a professional.

In [3]:
X = data.drop(columns="target_5y")
y = data['target_5y']

## Pipeline

👇 We are giving you the simple pipeline below

In [4]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# Preprocessing pipe
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaling', MinMaxScaler())
])

# Final pipe
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model_svm', SVC())
])

pipe

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('imputer', SimpleImputer()),
                                 ('scaling', MinMaxScaler())])),
                ('model_svm', SVC())])

In [5]:
pipe.get_params()

{'memory': None,
 'steps': [('preprocessing',
   Pipeline(steps=[('imputer', SimpleImputer()), ('scaling', MinMaxScaler())])),
  ('model_svm', SVC())],
 'verbose': False,
 'preprocessing': Pipeline(steps=[('imputer', SimpleImputer()), ('scaling', MinMaxScaler())]),
 'model_svm': SVC(),
 'preprocessing__memory': None,
 'preprocessing__steps': [('imputer', SimpleImputer()),
  ('scaling', MinMaxScaler())],
 'preprocessing__verbose': False,
 'preprocessing__imputer': SimpleImputer(),
 'preprocessing__scaling': MinMaxScaler(),
 'preprocessing__imputer__add_indicator': False,
 'preprocessing__imputer__copy': True,
 'preprocessing__imputer__fill_value': None,
 'preprocessing__imputer__missing_values': nan,
 'preprocessing__imputer__strategy': 'mean',
 'preprocessing__imputer__verbose': 0,
 'preprocessing__scaling__clip': False,
 'preprocessing__scaling__copy': True,
 'preprocessing__scaling__feature_range': (0, 1),
 'model_svm__C': 1.0,
 'model_svm__break_ties': False,
 'model_svm__cache_size

## Fine-Tuning

Our task is to assist the recruitment process of promising young players.  
The model should **limit false alarms as much as possible** to avoid recruiting players that will flop.

❓ **Fine-tune this pipeline so as to maximize your objective**

- Use the `scoring` metric appropriate for the task
- Grid Search for the optimal:
    - imputing `strategy`
    - `kernel`
    - regularization factor `C`


- Store your random search results in a `search`

In [25]:
from sklearn.model_selection import GridSearchCV

pipe.get_params()


grid_search = GridSearchCV(
    pipe, 
    param_grid={
        
        'preprocessing__imputer__strategy': ['mean','median'],
        'model_svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid','precomputed'],
        'model_svm__C':[1.0,10,100]
    },
    cv=5,
    scoring="precision",n_jobs=-1)

search = grid_search.fit(X, y)

pipe_pumped = grid_search.best_estimator_


/Users/guilhermecavalcantidesabarreto/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/guilhermecavalcantidesabarreto/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/guilhermecavalcantidesabarreto/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit

In [23]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solution',
    search = search
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/guilhermecavalcantidesabarreto/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/guilhermecavalcantidesabarreto/code/guipyc/data-tuning-pipeline
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_solution.py::TestSolution::test_cv_results PASSED             [100%]

============================== 1 passed in 0.42s ===============================


💯 You can commit your code:

git add tests/solution.pickle

git commit -m 'Completed solution step'

git push origin master


In [24]:
!git add tests/solution.pickle

!git commit -m 'Completed solution step'

!git push origin master

[master 420c8f0] Completed solution step
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 tests/solution.pickle
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 8 threads
Compressing objects: 100% (12/12), done.
Writing objects: 100% (13/13), 46.48 KiB | 5.16 MiB/s, done.
Total 13 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), done.
To github.com:guipyc/data-tuning-pipeline.git
 * [new branch]      master -> master


In [28]:
pipe_pumped

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                 ('scaling', MinMaxScaler())])),
                ('model_svm', SVC(kernel='poly'))])

## Export

Once you have built your optimal pipeline, export it as a pickle file

In [31]:
import pickle

with open("pipeline.pkl", "wb") as file:
    pickle.dump(pipe_pumped, file)

🏁 Congratulation. Don't forget to add, commit and push your notebook.